In [260]:
#!/usr/bin/env python
# encoding: utf-8

import tweepy #https://github.com/tweepy/tweepy
import csv

#Twitter API credentials
access_key = "954318739-o3d1RrR3DTO0rA3vf3RqyBdEUOFInILJcYA06JF7"
access_secret = "saQTFMe03E5tFFKzDfITIoLuy7wVlp0EKHeNozsWX1tnC"
consumer_key = "OD4OqJ8DWZBlxqX2c5iQoFT1R"
consumer_secret = "k0h6W8L05swc33fW21447Re5VFZkbSx2BcHr9wZiBmoJmF5f0y"

def get_all_tweets(screen_name):
#Twitter only allows access to a users most recent 3240 tweets with this method
#  
	#authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
	
	#initialize a list to hold all the tweepy Tweets
    alltweets=[]
	
	#make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)
	
	#save most recent tweets
    alltweets.extend(new_tweets)
	
	#save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
	
	#keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print("getting tweets before {}".format(oldest))
		
		#all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
		
		#save most recent tweets
        alltweets.extend(new_tweets)
		
		#update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
		
        print( "...{} tweets downloaded so far".format(len(alltweets)))
	
	#transform the tweepy tweets into a 2D array that will populate the csv	
    outtweets = [[tweet.id_str, tweet.created_at, tweet.text.encode("utf-8")] for tweet in alltweets]
	
	#write the csv	
    with open('%s_tweets.csv' % screen_name, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["id","created_at","text"])
        writer.writerows(outtweets)
    pass




if __name__ == '__main__':
	#pass in the username of the account you want to download
    get_all_tweets("imaginedragons")

getting tweets before 878770386585997311
...400 tweets downloaded so far
getting tweets before 837753999013007359
...599 tweets downloaded so far
getting tweets before 826884682436341762
...799 tweets downloaded so far
getting tweets before 747290524487294975
...996 tweets downloaded so far
getting tweets before 687368739961073669
...1196 tweets downloaded so far
getting tweets before 654401703920451587
...1394 tweets downloaded so far
getting tweets before 630897000171442175
...1594 tweets downloaded so far
getting tweets before 595751398119182335
...1794 tweets downloaded so far
getting tweets before 575849245544091648
...1994 tweets downloaded so far
getting tweets before 569219417898246143
...2194 tweets downloaded so far
getting tweets before 567832701992083455
...2392 tweets downloaded so far
getting tweets before 557703641164177407
...2592 tweets downloaded so far
getting tweets before 537312451097001983
...2792 tweets downloaded so far
getting tweets before 515259366698909695
.

In [1]:
import numpy as np
import pandas as pd

# stats
from scipy import stats
# Plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import math


%matplotlib inline

x = pd.read_csv('training.csv',sep=",",encoding='latin-1',names=[1,2,3,4,5,6])
x = x.drop([2,3,4,5],axis=1)
x = x.drop([0],axis=0)
neg_train = pd.DataFrame(x[x[1]==0])
pos_train = pd.DataFrame(x[x[1]==4])
X = pd.DataFrame(x)

In [2]:
Y = X.sample(frac=0.2).reset_index(drop=True)
# X.loc[np.arange(1,20),[1]]
Y.head()

,1,6
0,0,My fiance got me sick
1,0,@JaneBelinda yes tennis widow again - weds nig...
2,4,"@thiinkle looks damn sexy, more user feedback ..."
3,4,@GaneshaXi at least you didn't leave the lapto...
4,4,@shrek9 Point is those elements don't become C...


In [16]:
X.loc[np.arange(1,20),[1]]

,1,6
0,0,@ehhh_victoria Hey V!! I haven't talked to yo...
1,0,ugh work 3 to 11
2,4,@bmanley Meh! If a place could be considered h...
3,0,"@Carl_Zilyvich Not happening! :S mira, ahi tie..."
4,4,@donambridge why thanks for the compliment!!!


In [3]:
import re
def processTweet(tweet):
    with open("tweets.txt",'w') as lol:
        for line in x.loc[:,3]:
            line = re.sub(r'[^a-zA-Z]+$',' ', line)
            line = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+))','URL',line)
            line = re.sub(r'#([^\s]+)', r'\1', line)
            line = re.sub('@',' ',line)
            line = re.sub('\*\*+', '*', line)
            line = line.replace('\\n',' ')
            
            print(line,end="\n",file = lol)

In [4]:
def processTweet_one(tweet):
    line = re.sub(r'[^a-zA-Z]+$',' ', tweet)
    line = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+))','URL',line)
    line = re.sub(r'#([^\s]+)', r'\1', line)
    line = re.sub('@',' ',line)
    line = re.sub('\*\*+', '*', line)
    line = line.replace('\\n',' ')
    return line
    

In [263]:
processTweet(x)

In [264]:
# to download stop words
# import nltk
# nltk.download('stopwords')

In [5]:
loll = list()
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))
stopWords2 = list(stopWords)
stopWords2.append('url')


In [6]:
def getFeatureVector(tweet,stopWords):
    featureVector = []
    #split tweet into words
    words = tweet.split()
    for w in words:
        w = w.strip('\'"?,.')
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
        
        if(w in stopWords or val==None or w == 'URL' or w=='url'):
            continue
        else:
            featureVector.append(w.lower())
    return featureVector

In [7]:
features = list()
with open("tweets.txt",'r') as lol:
    for x in lol:
        print(x)
        w = getFeatureVector(x,stopWords2)
        features.append(w)



b'Evolve World Tour Night 20 - Montreal URL 

b'RT  them: Queer members of the Mormon community have found an unlikely ally in this  Imaginedragons frontman. URL 

b'Dan is gonna call whoever listens to Evolve on  iHeartRadio the most this weekend. iHeartAllAccess URL 

b'Tune in NOW at URL ImagineDragonsLive 

b'3 hours till we stream live on twitter.  tune in at URL  if you live in the U.S. or Canada. ImagineDragonsLive 

b'Evolve World Tour Night 19 - Quebec City URL 

b"to our friends outside those countries, you'll get to see it later, don't worry 

b'we are going to be streaming our concert from Montreal LIVE tonight in U.S./Canada at 9:30pm EST. go to URL to join 

b'see you soon, Tokyo. tickets on sale December 9 at 10am. URL 

b"RT  Cirque: Here\xe2\x80\x99s a behind the scenes look at a collaboration O did with  ImagineDragons' new music video \xe2\x80\x9cWhatever it Takes\xe2\x80\x9d\xe2\x9d\x97\xef\xb8\x8fURL 

b'Evolve World Tour Night 18 - New Jersey URL 

b'Evolve World 


b'Guy: i love imagine dragons can we take a pic Me: sure After pic Guy: thanks! Sex is on fire is my favorite jam btw great work Me 

b'what do you believe in? URL 

b'RT  TIME: 5 songs you need to listen to this week URL 

b"RT  billboard: Imagine Dragons, Big Sean &amp; more are  Spotify editors' picks for NewMusicFriday URL URL 

b'big ups to  Spotify for including Believer in New Music Friday.  Stream it right here: URL 

b'insomnia 

b'remember. we all make mistakes. URL 

b'\xf0\x9f\x8c\x 

b' RadioMadison  DanReynolds thanks for the love Madison! I see u standing up for good causes all the time. popular or not. keep being u Xxdr 

b'All people WELCOME. Every race. religion. culture. ethnicity. sex. EVERYONE. halsey  Zedd  Daya  macklemore\xe2\x80\xa6 URL 

b'making plans 

b'RT  DanielPlatzman: First things first 

b'BELIEVER  Vevo URL 

b'You made us a believer. Thank you. All of you. Xxid 

b' martoleision000 and englishes 

b'Nobody wins in that scenario  AbbeyChatwin 

b' i


b' Vevo is celebrating 100 million views on one of our music videos by inviting someone out to a shoot in Vegas: URL 

b'getting ready for the new year and we have some surprises in store these next couple months 

b'(and for those looking for a way to use their itunes gift cards today: SmokeAndMirrors URL 

b'Merry Christmas to the greatest fans and friends a band could ask for. can you name this dragon? URL 

b'RT  iTunesMusic: SMOKEANDMIRRORS is coming. Two new  Imaginedragons tracks are yours with pre-order. URL URL 

b'GOLD URL URL 

b'UNLV 

b'third and final rule of following directions: the early bird catches the worm. and there is only one worm 

b'second rule of following directions: make sure your navigation tools are up to par 

b"'I'm So Sorry' the album isn't till 2/17. but pre-order the U.S. deluxe version right here: URL URL 

b'first rule of following directions: be precise 

b'pre-ordered our upcoming album Smoke + Mirrors yet? 2 instant grat songs when you do: URL U

In [272]:
features


[['world', 'tour', 'night', 'montreal'],
 ['queer',
  'members',
  'mormon',
  'community',
  'found',
  'unlikely',
  'ally',
  'imaginedragons',
  'frontman'],
 ['gonna',
  'call',
  'whoever',
  'listens',
  'evolve',
  'iheartradio',
  'weekend',
  'iheartallaccess'],
 ['now', 'imaginedragonslive'],
 ['hours',
  'till',
  'stream',
  'live',
  'twitter',
  'tune',
  'live',
  'canada',
  'imaginedragonslive'],
 ['world', 'tour', 'night', 'quebec', 'city'],
 ['friends', 'outside', 'countries', 'get', 'see', 'later', 'worry'],
 ['going',
  'streaming',
  'concert',
  'montreal',
  'live',
  'tonight',
  'est',
  'go',
  'join'],
 ['soon', 'tokyo', 'tickets', 'sale', 'december'],
 ['behind',
  'scenes',
  'look',
  'collaboration',
  'o',
  'imaginedragons',
  'new',
  'music',
  'video'],
 ['world', 'tour', 'night', 'new', 'jersey'],
 ['world', 'tour', 'night', 'brooklyn'],
 ['world', 'tour', 'night', 'cincinnati'],
 ['world', 'tour', 'night', 'detroit'],
 ['world', 'tour', 'night', 

In [8]:
tweets = []
for index, row in Y.iterrows():
    sentiment = row[1]
    tweet = row[6]
    processedTweet = processTweet_one(tweet)
    featureVector = getFeatureVector(processedTweet, stopWords)
    tweets.append((featureVector, sentiment));

In [9]:
tweets

[(['my', 'fiance', 'got', 'sick'], 0),
 (['janebelinda',
   'yes',
   'tennis',
   'widow',
   'weds',
   'night',
   'louis',
   'asleep',
   'hot',
   'thinks',
   'teeth',
   'again',
   'red',
   'vino',
   'time',
   'x'],
  0),
 (['thiinkle', 'looks', 'damn', 'sexy', 'user', 'feedback', 'pending'], 4),
 (['ganeshaxi', 'least', 'leave', 'laptop'], 4),
 (['shrek9',
   'point',
   'elements',
   'become',
   'cabinet',
   'ministers',
   'push',
   'forward',
   'regressive',
   'cong'],
  4),
 (['mckmama', 'so', 'i', 'read', 'whe', 'i', 'get', 'home', 'almost'], 4),
 (['casndra',
   'it',
   'veryyyyyyy',
   'stressful',
   'i',
   'dunno',
   'if',
   'really',
   'wanna',
   'hear',
   'direct',
   'message'],
  4),
 (['i', 'hate', 'watching', 'news', 'why', 'awful', 'things', 'happen'], 0),
 (['kataylor007',
   'aww',
   'sorry',
   'i',
   'hope',
   'jordanknight',
   'help',
   'cheer',
   'u'],
  0),
 (['praying',
   'project',
   'alaska',
   'right',
   'looks',
   'like',

In [22]:
import sys
import os
import time
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import classification_report

classes = [0,4]
train_data = []
train_labels = []
test_data = []
test_labels = []

train_data, test_data, train_labels, test_labels = train_test_split(Y.loc[np.arange(0,200),6], Y.loc[np.arange(0,200),1], test_size=0.33, random_state=42)
    # Create feature vectors
vectorizer = TfidfVectorizer(min_df=5,
                                 max_df = 0.8,
                                 sublinear_tf=True,
                                 use_idf=True)
train_vectors = vectorizer.fit_transform(train_data)
test_vectors = vectorizer.transform(test_data)

    # Perform classification with SVM, kernel=rbf
# classifier_rbf = svm.SVC()
# t0 = time.time()
# classifier_rbf.fit(train_vectors, train_labels)
# t1 = time.time()
# prediction_rbf = classifier_rbf.predict(test_vectors)
# t2 = time.time()
# time_rbf_train = t1-t0
# time_rbf_predict = t2-t1

    # Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(train_vectors, train_labels)
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1

#     # Perform classification with SVM, kernel=linear
# classifier_liblinear = svm.LinearSVC()
# t0 = time.time()
# classifier_liblinear.fit(train_vectors, train_labels)
# t1 = time.time()
# prediction_liblinear = classifier_liblinear.predict(test_vectors)
# t2 = time.time()
# time_liblinear_train = t1-t0
# time_liblinear_predict = t2-t1

    # Print results in a nice table
# print("Results for SVC(kernel=rbf)")
# print("Training time: %fs; Prediction time: %fs" % (time_rbf_train, time_rbf_predict))
# print(classification_report(test_labels, prediction_rbf))
print("Results for SVC(kernel=linear)")
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
print(classification_report(test_labels, prediction_linear))
# print("Results for LinearSVC()")
# print("Training time: %fs; Prediction time: %fs" % (time_liblinear_train, time_liblinear_predict))
# print(classification_report(test_labels, prediction_liblinear))

Results for SVC(kernel=linear)
Training time: 0.000000s; Prediction time: 0.000000s
             precision    recall  f1-score   support

          0       0.55      0.77      0.64        35
          4       0.53      0.29      0.38        31

avg / total       0.54      0.55      0.52        66



In [12]:
test_data

95     just sending a twitter from the club! i twitte...
15     The and of Egoland. http://egoland.dk/ You don...
30     Offically done with Spanish and theature!  Jus...
158    So bummed out right now that after tonight I d...
128                     I want a pet Alpaca called Jake 
115    I love horses haha inside joke haha its fun to...
69     My head hurts   why didn't anyone tell me i wa...
170    @alpinesucre I can't get any sleep  http://myl...
174    @fan4lyf i agree with you girl ! i wish that m...
45                              heyree parkooo is here! 
66     Can't procrastinate any more - tho ought to go...
182    getting ready for Sauna Open Air.... but the w...
165    @YorkshireTeaIan  Probably need your brolly Ia...
78     @thanktank Hmm? Who are you na pala? Sorry. Ha...
186    First day at FSFE's General Assembly. I feel s...
177    @Pepsimo seems ok. contractions this morn, did...
56           .. going for a run now!  .. cya guys later!
152    Have to clean install 3.

In [103]:
m = ['Queer members of the Mormon community have found an unlikely ally in this  Imaginedragons frontman. URL '

,'Dan is gonna call whoever listens to Evolve on  iHeartRadio the most this weekend. iHeartAllAccess URL' 

,'Tune in NOW at URL ImagineDragonsLive '

,'3 hours till we stream live on twitter.  tune in at URL  if you live in the U.S. or Canada. ImagineDragonsLive' 

,'Evolve World Tour Night 19 - Quebec City URL'

,'to our friends outside those countries, youll get to see it later, dont worry' 

,'we are  going  soon on sale to  be streaming our concert from Montreal LIVE tonight in U.S./Canada at 9:30pm EST. go to URL to join'

,'see you soon, Tokyo. tickets on sale December 9 at 10am.depressed stress sad URL',
    "see you soon","happy, soon not to see you live"]

In [104]:
m

['Queer members of the Mormon community have found an unlikely ally in this  Imaginedragons frontman. URL ',
 'Dan is gonna call whoever listens to Evolve on  iHeartRadio the most this weekend. iHeartAllAccess URL',
 'Tune in NOW at URL ImagineDragonsLive ',
 '3 hours till we stream live on twitter.  tune in at URL  if you live in the U.S. or Canada. ImagineDragonsLive',
 'Evolve World Tour Night 19 - Quebec City URL',
 'to our friends outside those countries, youll get to see it later, dont worry',
 'we are  going  soon on sale to  be streaming our concert from Montreal LIVE tonight in U.S./Canada at 9:30pm EST. go to URL to join',
 'see you soon, Tokyo. tickets on sale December 9 at 10am.depressed stress sad URL',
 'see you soon',
 'happy, soon not to see you live']

In [105]:
test_vectors = vectorizer.transform(m)
  # Perform classification with SVM, kernel=linear

prediction_linear = classifier_linear.predict(test_vectors)

print(prediction_linear)

[0 0 4 4 0 4 0 4 4 0]


In [106]:
X.head()

,1,6
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
5,0,@Kwesidei not the whole crew


In [20]:
test_vectors

<66x63 sparse matrix of type '<class 'numpy.float64'>'
	with 255 stored elements in Compressed Sparse Row format>

In [252]:
lo = "fuck you   43242   lol all url"
x = getFeatureVector(lo,stopWords2)
# lo = lo.split()
# # for x in l:
# #     if re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", x):
# #         print("string")
# #     else:
# #         print("lol")
# for x in stopWords :
#     if(x in lo):
#         print("lol")
#     else:
#         print("baka")


In [253]:
x

['fuck', 'lol']